In [1]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

In [2]:
ko_text = "이것은 m2m모델로 만든 다국어 번역기 입니다."
chinese_text = "生活就像一盒巧克力。"

model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

# 한국어를 영어로 번역하기
tokenizer.src_lang = "ko"
encoded_hi = tokenizer(ko_text, return_tensors="pt")
generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.get_lang_id("en"))
result1=tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(result1)
# => "La vie est comme une boîte de chocolat."

# translate Chinese to English
tokenizer.src_lang = "zh"
encoded_zh = tokenizer(chinese_text, return_tensors="pt")
generated_tokens = model.generate(**encoded_zh, forced_bos_token_id=tokenizer.get_lang_id("ko"))
result2=tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(result2)
# => "Life is like a box of chocolate."


['This is a multi-language translator made with the m2m model.']
['인생은 초콜릿 상자와 같다.']


In [3]:
import gradio as gr
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

In [4]:
# 모델 불러오기
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

# 번역 함수
def translate(text, source_lang, target_lang):
    if not text.strip():
        return ""
    tokenizer.src_lang = source_lang
    encoded = tokenizer(text, return_tensors="pt")
    generated_tokens = model.generate(
        **encoded,
        forced_bos_token_id=tokenizer.get_lang_id(target_lang)
    )
    result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    return result[0]


# Gradio UI
with gr.Blocks(title="M2M100 번역기") as demo:

    gr.Markdown("## 🌐 M2M100 다국어 번역기")

    with gr.Row():

        # --- LEFT SIDE ---
        with gr.Column():
            src_lang = gr.Dropdown(
                ["한국어 (ko)", "영어 (en)", "중국어 (zh)","일본어(ja)","스페인어(es)"],
                value="한국어 (ko)",
                label="입력 언어"
            )

            input_text = gr.Textbox(
                label="입력 텍스트",
                placeholder="번역할 문장을 입력하세요.",
                lines=6
            )

        # --- RIGHT SIDE ---
        with gr.Column():
            tgt_lang = gr.Dropdown(
                ["한국어 (ko)", "영어 (en)", "중국어 (zh)","일본어(ja)","스페인어(es)"],
                value="영어 (en)",
                label="번역 언어"
            )

            output_text = gr.Textbox(
                label="번역 결과",
                lines=6
            )

    # 번역 버튼
    translate_btn = gr.Button("번역하기")

    # 버튼 클릭 동작
    def on_translate(text, src, tgt):
        # (ko) 형태에서 언어 코드만 추출
        src_code = src.split("(")[-1].replace(")", "")
        tgt_code = tgt.split("(")[-1].replace(")", "")
        return translate(text, src_code, tgt_code)

    translate_btn.click(
        on_translate,
        inputs=[input_text, src_lang, tgt_lang],
        outputs=output_text
    )

demo.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [10]:
demo.close()

Closing server running on port: 7860


## 한영-영한 번역 후 음성으로 읽어주는 서비스

### 영어를 읽어주는 모델

In [6]:
!pip install -q kokoro>=0.9.2 soundfile
# !apt-get -qq -y install espeak-ng > /dev/null 2>&1
from kokoro import KPipeline
from IPython.display import display, Audio
import soundfile as sf
import torch
pipeline = KPipeline(lang_code='a')
text ='Though seen as one of the most conservative Democrats in the U.S. House, Cuellar is not switching parties in 2026. In a post to social media on Dec. 7, Trump slammed the congressman for "continuing to work with the same Radical Left Scum."'
generator = pipeline(text, voice='af_heart')
for i, (gs, ps, audio) in enumerate(generator):
    print(i, gs, ps)
    display(Audio(data=audio, rate=24000, autoplay=i==0))
    sf.write(f'{i}.wav', audio, 24000)


/home/user/miniforge3/envs/hug/lib/python3.10/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
/home/user/miniforge3/envs/hug/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


0 Though seen as one of the most conservative Democrats in the U.S. House, Cuellar is not switching parties in 2026. In a post to social media on Dec. 7, Trump slammed the congressman for "continuing to work with the same Radical Left Scum." ðˌO sˈin æz wˈʌn ʌv ðə mˈOst kənsˈɜɹvəTɪv dˈɛməkɹˌæts ɪn ðə jˌuˈɛs hˈWs, kjˈuələɹ ɪz nˌɑt swˈɪʧɪŋ pˈɑɹTiz ɪn twˈɛnti twˈɛnti sˈɪks. ɪn ɐ pˈOst tə sˈOʃᵊl mˈidiə ˌɔn dˈɛk. sˈɛvən, tɹˈʌmp slˈæmd ðə kˈɑŋɡɹəsmən fɔɹ “kəntˈɪnjuɪŋ tə wˈɜɹk wɪð ðə sˈAm ɹˈædəkᵊl lˈɛft skˈʌm.”


### 한국어를 읽어주는 모델

In [8]:
# !pip install uroman

In [7]:
from transformers import VitsModel, AutoTokenizer
import torch
from uroman import Uroman
import soundfile as sf

model = VitsModel.from_pretrained("facebook/mms-tts-kor")
tokenizer = AutoTokenizer.from_pretrained("facebook/mms-tts-kor")

text = "이것은 허깅페이스에서 받은 모델로 만든 한국어 읽어주는 프로그램입니다."
uroman=Uroman()
roman_text=uroman.romanize_string(text)

inputs = tokenizer(roman_text, return_tensors="pt")

with torch.no_grad():
    output = model(**inputs).waveform

# 5. shape 정리
waveform = output.squeeze(0).cpu().numpy()   # shape (N,)

# 6. Notebook에서 바로 재생
display(Audio(waveform, rate=model.config.sampling_rate))
    
sf.write("korean_voice.wav",waveform,24000)


In [9]:
import gradio as gr
import torch
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
from transformers import AutoTokenizer, VitsModel
from kokoro import KPipeline
from uroman import Uroman
import soundfile as sf
import numpy as np

# -------------------------------------------
# 1) 번역 모델 로드 (M2M100)
# -------------------------------------------
model_trans = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer_trans = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

def translate(text, source_lang, target_lang):
    if not text.strip():
        return ""
    tokenizer_trans.src_lang = source_lang
    encoded = tokenizer_trans(text, return_tensors="pt")
    generated_tokens = model_trans.generate(
        **encoded,
        forced_bos_token_id=tokenizer_trans.get_lang_id(target_lang)
    )
    result = tokenizer_trans.batch_decode(generated_tokens, skip_special_tokens=True)
    return result[0]


# -------------------------------------------
# 2) 영어 TTS (Kokoro)
# -------------------------------------------
pipeline_en = KPipeline(lang_code="a")   # 영어 음성
def tts_english(text):
    generator = pipeline_en(text, voice="af_heart")
    for _, _, audio in generator:
        return (24000, audio)  # sample_rate, waveform


# -------------------------------------------
# 3) 한국어 TTS (MMS-TTS-KOR)
# -------------------------------------------
model_ko_tts = VitsModel.from_pretrained("facebook/mms-tts-kor")
tokenizer_ko_tts = AutoTokenizer.from_pretrained("facebook/mms-tts-kor")
uroman = Uroman()

def tts_korean(text):
    roman = uroman.romanize_string(text)
    inputs = tokenizer_ko_tts(roman, return_tensors="pt")

    with torch.no_grad():
        audio = model_ko_tts(**inputs).waveform

    waveform = audio.squeeze(0).cpu().numpy()
    return (model_ko_tts.config.sampling_rate, waveform)


# -------------------------------------------
# 4) 통합 처리 함수
# -------------------------------------------
def process(text, src, tgt):
    # 언어 코드 추출
    src_code = src.split("(")[-1].replace(")", "")
    tgt_code = tgt.split("(")[-1].replace(")", "")

    # 1) 번역
    translated = translate(text, src_code, tgt_code)

    # 2) 음성 변환
    if tgt_code == "en":
        sr, audio = tts_english(translated)
    elif tgt_code == "ko":
        sr, audio = tts_korean(translated)
    else:
        # 한국어/영어 외 언어 → 음성 미지원
        return translated, None

    return translated, (sr, audio)


# -------------------------------------------
# 5) Gradio UI
# -------------------------------------------
with gr.Blocks(title="번역 + 음성 합성기") as demo:

    gr.Markdown("## 🌐 다국어 번역 + 음성 합성 프로그램")

    with gr.Row():

        # 입력 부분
        with gr.Column():
            src_lang = gr.Dropdown(
                ["한국어 (ko)", "영어 (en)"],
                value="한국어 (ko)",
                label="입력 언어"
            )

            input_text = gr.Textbox(
                label="입력 텍스트",
                placeholder="번역할 문장을 입력하세요.",
                lines=6
            )

        # 출력 부분
        with gr.Column():
            tgt_lang = gr.Dropdown(
                ["한국어 (ko)", "영어 (en)"],
                value="영어 (en)",
                label="번역 언어"
            )

            output_text = gr.Textbox(
                label="번역 결과",
                lines=6
            )

            output_audio = gr.Audio(label="음성 출력", type="numpy")

    translate_btn = gr.Button("번역 + 음성 생성")

    translate_btn.click(
        process,
        inputs=[input_text, src_lang, tgt_lang],
        outputs=[output_text, output_audio]
    )

demo.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [11]:
import gradio as gr
import torch
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
from transformers import AutoTokenizer, VitsModel
from kokoro import KPipeline
import numpy as np

# -------------------------------------------
# 1) 번역 모델 로드 (M2M100)
# -------------------------------------------
model_trans = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer_trans = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

def translate(text, source_lang, target_lang):
    if not text.strip():
        return ""
    tokenizer_trans.src_lang = source_lang
    encoded = tokenizer_trans(text, return_tensors="pt")
    generated_tokens = model_trans.generate(
        **encoded,
        forced_bos_token_id=tokenizer_trans.get_lang_id(target_lang)
    )
    result = tokenizer_trans.batch_decode(generated_tokens, skip_special_tokens=True)
    return result[0]


# -------------------------------------------
# 2) 영어 TTS (Kokoro)
# -------------------------------------------
pipeline_en = KPipeline(lang_code="a")   # 영어 음성
def tts_english(text):
    generator = pipeline_en(text, voice="af_heart")
    for _, _, audio in generator:
        return (24000, audio)  # sample_rate, waveform


# -------------------------------------------
# 3) 한국어 TTS (MMS-TTS-KOR) — Uroman 제거 버전
# -------------------------------------------
model_ko_tts = VitsModel.from_pretrained("facebook/mms-tts-kor")
tokenizer_ko_tts = AutoTokenizer.from_pretrained("facebook/mms-tts-kor")

def tts_korean(text):
    # Uroman 변환 없이 바로 한국어 넣기
    inputs = tokenizer_ko_tts(text, return_tensors="pt")

    with torch.no_grad():
        audio = model_ko_tts(**inputs).waveform

    waveform = audio.squeeze(0).cpu().numpy()
    return (model_ko_tts.config.sampling_rate, waveform)


# -------------------------------------------
# 4) 통합 처리 함수
# -------------------------------------------
def process(text, src, tgt):
    # 언어 코드 추출
    src_code = src.split("(")[-1].replace(")", "")
    tgt_code = tgt.split("(")[-1].replace(")", "")

    # 1) 번역
    translated = translate(text, src_code, tgt_code)

    # 2) 음성 변환
    if tgt_code == "en":
        sr, audio = tts_english(translated)
    elif tgt_code == "ko":
        sr, audio = tts_korean(translated)
    else:
        # 한국어/영어 외 언어 → 음성 미지원
        return translated, None

    return translated, (sr, audio)


# -------------------------------------------
# 5) Gradio UI
# -------------------------------------------
with gr.Blocks(title="번역 + 음성 합성기") as demo:

    gr.Markdown("## 🌐 다국어 번역 + 음성 합성 프로그램")

    with gr.Row():

        # 입력 부분
        with gr.Column():
            src_lang = gr.Dropdown(
                ["한국어 (ko)", "영어 (en)"],
                value="한국어 (ko)",
                label="입력 언어"
            )

            input_text = gr.Textbox(
                label="입력 텍스트",
                placeholder="번역할 문장을 입력하세요.",
                lines=6
            )

        # 출력 부분
        with gr.Column():
            tgt_lang = gr.Dropdown(
                ["한국어 (ko)", "영어 (en)"],
                value="영어 (en)",
                label="번역 언어"
            )

            output_text = gr.Textbox(
                label="번역 결과",
                lines=6
            )

            output_audio = gr.Audio(label="음성 출력", type="numpy")

    translate_btn = gr.Button("번역 + 음성 생성")

    translate_btn.click(
        process,
        inputs=[input_text, src_lang, tgt_lang],
        outputs=[output_text, output_audio]
    )

demo.launch()



/home/user/miniforge3/envs/hug/lib/python3.10/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
/home/user/miniforge3/envs/hug/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
